In [1]:
!pip install pandas -q

import pandas as pd

# File paths
train_file = "./data_generation/train_data_min_max_input_half_lin_half_normal.csv"
test_file = "./data_generation/test_data_min_max_input_half_lin_half_normal.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Display the first few rows of each dataset
print("Train Data:")
print(train_df.info())

print("\nTest Data:")
print(test_df.info())



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  800000 non-null  float64
 1   Series2  800000 non-null  float64
 2   Series3  800000 non-null  float64
 3   Label    800000 non-null  float64
dtypes: float64(4)
memory usage: 24.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  200000 non-null  float64
 1   Series2  200000 non-null  float64
 2   Series3  200000 non-null  float64
 3   Label    200000 non-null  float64
dtypes: float64(4)
memory usage: 6.1 MB
None


In [ ]:
# install sinabs
!pip install sinabs -q


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.inputval = dataframe["Series1"]
        self.mean = dataframe["Series2"]
        self.std = dataframe["Series3"]
        self.labels = dataframe["Label"]

    def __len__(self):
        return len(self.inputval)

    def __getitem__(self, idx):
        inputval = self.inputval.iloc[idx]
        mean = self.mean.iloc[idx]
        std = self.std.iloc[idx]
        labels = self.labels.iloc[idx]
        return inputval, mean, std, labels
    
# Create the datasets
train_dataset = DataFrameDataset(train_df)
test_dataset = DataFrameDataset(test_df)


In [4]:
#import numpy

#num_bins = 4/0.01

#min_val = float(train_dataset.labels.min())
#max_val = float(train_dataset.labels.max())

#bins_train = torch.linspace(min_val, max_val, int(num_bins + 1))
#bin_indices_train = numpy.digitize(train_dataset.labels, bins_train, right=True)
##print(bin_indices_train)
#bin_counts = torch.bincount(torch.tensor(bin_indices_train))

# plot bin_counts
#import matplotlib.pyplot as plt
#plt.hist(bin_counts)
#plt.show()
#bin_counts

In [5]:
# We calculate the weights for each bin

#bin_weights = 1.0 / (bin_counts + 1e-6)  # Inverse of bin counts
#plt.hist(bin_weights)
#plt.show()
#print(len(bin_weights))
#print("Summed: ", sum(bin_weights))
#sample_weights = bin_weights[bin_indices]  # Assign weights to each sample

In [6]:
#len(sample_weights)

In [7]:
#from torch.utils.data import WeightedRandomSampler

#sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)


In [8]:
# Encode input data


train_loader = DataLoader(train_dataset, batch_size=64) #, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=64) #, shuffle=True)


In [9]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [10]:
import torch

TRAINING_PREFIX = "2x256hidden_SGD_train_data_final"

ann = nn.Sequential(
    nn.Linear(3, 256),  # Input layer: 3 features (mu, sigma, x)
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # first hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # second hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 1)    # Output layer: single value for f(x; mu, sigma)
)

#ann.load_state_dict(torch.load("largerer_ann_new_data4.pth"))

ann.to(device)

total_params = sum(p.numel() for p in ann.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params}")


Total Parameters: 132865


In [11]:
from sinabs.from_torch import from_model

num_time_steps_per_sample = 100

sinabs_model = from_model(ann, input_shape=(3,), add_spiking_output=True, synops=False, num_timesteps=num_time_steps_per_sample)
sinabs_model

Network(
  (spiking_model): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (spike_output): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
  )
  (analog_model): Sequential(
    (0): Linear(in_features=3, out_features

In [12]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from torcheval.metrics import R2Score


def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Eval Loss', line=dict(color='orange')))

    # Configure layout
    fig.update_layout(title='Training and Evaluation Losses',
                    xaxis_title='Epoch',
                    yaxis_title='Loss',
                    template='plotly_dark')

    # Display the figure widget
    display(fig)
    return fig

def update_loss_plot(fig, train_loss, eval_loss, from_epoch=0):
    if from_epoch != 0:
        with fig.batch_update():
            fig.data[0].x = list(range(from_epoch, len(train_loss)))
            fig.data[0].y = train_loss[from_epoch:]
            fig.data[1].x = list(range(from_epoch, len(eval_loss)))
            fig.data[1].y = eval_loss[from_epoch:]
    elif len(train_loss) < 30:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss)))
            fig.data[0].y = train_loss
            fig.data[1].x = list(range(len(eval_loss)))
            fig.data[1].y = eval_loss
    else:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss) - 30, len(train_loss)))
            fig.data[0].y = train_loss[-30:]
            fig.data[1].x = list(range(len(eval_loss) - 30, len(eval_loss)))
            fig.data[1].y = eval_loss[-30:]        
    

import numpy as np
def gaussian_probability(x, y, z):
    return (1 / (np.sqrt(2 * np.pi) * z)) * np.exp(-((x - y) ** 2) / (2 * z ** 2))

from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def do_epoch(loader, model, optimizer, device, n_epochs: int, current_epoch: int, train: bool = False,):
    if train:
        model.train()
    else:
        model.eval()
    epoch_loss = 0
    metric = R2Score(device=device)
    for inputvals, means, stds, labels in tqdm(loader, desc=f"{'Epoch' if train else 'Eval Epoch'} {current_epoch+1}/{n_epochs}"):
        inputs = torch.stack((inputvals, means, stds), dim=1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        metric.update(outputs.squeeze(), labels)
        if train:
            optimizer.zero_grad()
            loss = F.mse_loss(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
        else:
            loss = F.mse_loss(outputs.squeeze(), labels.float())
        epoch_loss += loss.item()
    return epoch_loss, metric.compute()

In [ ]:
ann.train()

N_EPOCHS = 500
LR = 0.01
SAVE_EVERY = 50

# Create subfolder for this loop
import os
import shutil

if os.path.exists(f"./results/{TRAINING_PREFIX}"):
    shutil.rmtree(f"./results/{TRAINING_PREFIX}")
os.makedirs(f"./results/{TRAINING_PREFIX}")

#optim = torch.optim.Adam(ann.parameters(), lr=LR)
optim = torch.optim.SGD(ann.parameters(), lr=LR)

last_loss = 0
last_eval_loss = 0

epoch_loss_list = []
eval_loss_list = [] 

fig = create_loss_plot()

for epoch in range(N_EPOCHS):
    if (epoch + 1) % 30 == 0:
        clear_output(wait=True)
        fig = create_loss_plot()
        update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    
    epoch_loss, train_R2_score = do_epoch(train_loader, ann, optim, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=True)
    eval_epoch_loss, eval_R2_score = do_epoch(test_loader, ann, None, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=False)
    eval_loss_list.append(eval_epoch_loss/len(test_loader))
    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(train_loader)}, Diff: {epoch_loss/len(train_loader) - last_loss}, Eval Loss: {eval_epoch_loss/len(test_loader)}, Diff Eval: {eval_epoch_loss/len(test_loader) - last_eval_loss}, Train R2 Score: {train_R2_score}, Eval R2 Score: {eval_R2_score}")
    last_loss = epoch_loss/len(train_loader)
    last_eval_loss = eval_epoch_loss/len(test_loader)
    epoch_loss_list.append(epoch_loss/len(train_loader))
    update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    if epoch % SAVE_EVERY == 0:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth")
    elif epoch == N_EPOCHS - 1:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth") 

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': 'c8192afd-3a2c-40b1-b096-7ab7ee78278b',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': 'e89382c7-a54e-4afe-af7e-640ebd77d834',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

Epoch 1/500:   0%|          | 0/12500 [00:00<?, ?it/s]

In [ ]:
#torch.save(ann.state_dict(), "largerer_ann_new_data5.pth")


In [ ]:
fig = create_loss_plot()
update_loss_plot(fig, epoch_loss_list, eval_loss_list, 30)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': 'eca93ac4-9486-4bc9-a119-8d0756c8c38e',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': '6aa530ca-8544-4b2c-9fc5-cbad0bc7306c',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

In [ ]:
ann.eval()
ann(torch.tensor([[0.5, 0.5, 0.5]]).to(device)).item()


0.7964390516281128

In [ ]:
gaussian_probability(0.5, 0.5, 0.5)

np.float64(0.7978845608028654)

In [ ]:
ann(torch.tensor([[0.1, 0.1, 0.1]]).to(device)).item()

3.990462064743042

In [ ]:
gaussian_probability(0.1, 0.1, 0.1)

np.float64(3.989422804014327)

In [ ]:
ann(torch.tensor([[0.9, 0.9, 0.9]]).to(device)).item()

0.4476521611213684

In [ ]:
gaussian_probability(0.9, 0.9, 0.9)

np.float64(0.44326920044603635)

In [ ]:
ann(torch.tensor([[0.9, 0.7, 0.1]]).to(device)).item()

0.5147292017936707

In [ ]:
gaussian_probability(0.9, 0.7, 0.1)

np.float64(0.53990966513188)

In [ ]:
ann(torch.tensor([[0.643, 0.7, 0.1]]).to(device)).item()

3.3867263793945312

In [ ]:
gaussian_probability(0.643, 0.7, 0.1)

np.float64(3.3912431320419234)

In [ ]:
ann(torch.tensor([[4, 0.3, 0.2]]).to(device)).item() # Not really generalized to computation of f(x; mu, sigma), but learned kind of in the ranges the data existed in

-5.334615707397461e-06

In [ ]:
gaussian_probability(4, 0.3, 0.2)

np.float64(9.57716245835995e-75)

In [ ]:
train_dataset.mean.unique()

array([ 0.41253237, -0.72986496, -1.61839622,  1.36453014,  2.57039399,
       -1.5549297 , -2.57039399,  0.1586663 ,  0.47599889,  1.42799666,
        1.11066407, -1.04719755,  0.28559933,  2.2530614 , -3.07812614,
        2.18959488, -3.01465962, -2.63386051,  1.23759711,  2.50692747,
        0.856798  , -1.36453014, -2.2530614 , -2.9511931 , -1.68186273,
        2.37999443, -0.53946541, -1.23759711, -1.17413059, -1.42799666,
        1.17413059, -0.66639844,  0.60293192, -0.28559933, -1.93572881,
        1.68186273,  2.76079354, -2.76079354,  1.49146318, -2.69732703,
       -0.41253237, -2.12612836, -0.09519978, -0.34906585,  2.31652792,
       -1.99919533, -0.03173326, -2.50692747,  2.06266184,  3.01465962,
       -2.31652792,  3.14159265, -1.87226229,  0.92026451,  1.5549297 ,
        0.53946541,  0.79333148,  0.03173326, -0.92026451,  0.09519978,
        2.82426006,  1.74532925,  2.12612836, -2.44346095, -0.98373103,
        1.04719755,  1.61839622,  2.44346095,  1.30106362,  0.22

In [ ]:
train_dataset.std.unique()

array([4.34703637, 1.5364976 , 1.03684626, 5.84599039, 3.66001578,
       6.15827247, 0.1       , 5.9084468 , 0.34982567, 0.28736925,
       6.22072889, 3.47264653, 0.66210776, 2.72316952, 3.59755937,
       2.22351819, 0.97438984, 1.72386685, 0.16245642, 3.53510295,
       4.53440562, 5.1589698 , 3.78492862, 2.91053878, 3.03545161,
       0.78702059, 3.84738503, 3.90984145, 2.78562594, 4.40949279,
       4.78423129, 2.97299519, 5.22142621, 2.09860535, 1.09930268,
       5.78353397, 1.22421551, 1.9112361 , 4.03475429, 4.84668771,
       6.09581606, 3.7224722 , 4.72177488, 5.47125188, 2.34843102,
       5.72107755, 5.59616472, 0.84947701, 1.78632327, 4.65931846,
       3.97229787, 4.15966712, 3.16036444, 5.5337083 , 0.72456417,
       4.90914413, 5.65862113, 1.16175909, 1.34912834, 3.09790803,
       0.59965134, 5.97090322, 1.59895401, 3.22282086, 2.66071311,
       4.22212354, 2.03614893, 2.53580027, 1.66141043, 2.41088744,
       0.41228209, 3.28527728, 1.41158476, 2.16106177, 6.28318

In [ ]:
# std 1.25 and mean -3.04
ann(torch.tensor([[-2, -3.04, 1.25]]).to(device)).item()

0.22803238034248352

In [ ]:
gaussian_probability(-2, -3.04, 1.25)

np.float64(0.22578002723581)

In [ ]:
-3.04 - 3 * 1.25

-6.79